<a href="https://colab.research.google.com/github/eduardobarros13/investsystem/blob/master/IQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
from datetime import date
from datetime import timedelta
import pandas as pd
from IPython.display import display, HTML
from bs4 import BeautifulSoup
import requests
import json
import plotly.graph_objects as go
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
feriados = {}
feriados[datetime.datetime(2020, 2, 24).strftime("%d/%m/%Y")] = True
feriados[datetime.datetime(2020, 2, 25).strftime("%d/%m/%Y")] = True
feriados[datetime.datetime(2020, 4, 10).strftime("%d/%m/%Y")] = True
feriados[datetime.datetime(2020, 4, 21).strftime("%d/%m/%Y")] = True
feriados[datetime.datetime(2020, 5, 1).strftime("%d/%m/%Y")] = True
feriados[datetime.datetime(2020, 7, 17).strftime("%d/%m/%Y")] = True
feriados[datetime.datetime(2020, 9, 7).strftime("%d/%m/%Y")] = True
feriados[datetime.datetime(2020, 10, 12).strftime("%d/%m/%Y")] = True
feriados[datetime.datetime(2020, 11, 2).strftime("%d/%m/%Y")] = True

### ESCOLHER A AÇÃO PRINCIPAL

In [3]:
petro = {"nome": "PETROLEO BRASILEIRO S.A. PETROBRAS", "ticker":'PETR4', "letra": "P", "tipo":"PN", "aluguel":"PETROLEO%20BRASIL"}
vara = {"nome": "VIA VAREJO S.A.", "ticker":'VVAR3', "letra": "V", "tipo":"ON"}
itub = {"nome": "ITAU UNIBANCO HOLDING S.A.","ticker":'ITUB4', "letra": "I", "tipo":"PN"}
bradesco = {"nome": "BCO BRADESCO S.A.","ticker":'BBDC4', "letra": "B", "tipo":"PN"}
bb = {"nome": "BCO BRASIL S.A.", "ticker":'BBAS3',"letra": "B", "tipo":"ON"}
conga = {"nome": "COGNA", "ticker":'COGN3',"letra": "C", "tipo":"ON"}
irb = {"nome": "IRB", "ticker":'IRBR3',"letra": "I", "tipo":"ON"}
vale = {"nome": "VALE", "ticker":'VALE3',"letra": "V", "tipo":"ON"}
empresa = petro

### Funções

In [4]:
def isFeriado(data):
  return feriados.get(data.strftime("%d/%m/%Y"))

##### Calcula a data de vencimento do próximo strike
def getDataVencimento(data):
    dia1Mes = data - timedelta(data.day-1)
    weekDay1 = dia1Mes.weekday()
    
    contador = 0
    diaVencimento = 0
    
    while(contador < 3):
        diaVencimento += 1        
        if(weekDay1 % 7) == 0:
            contador += 1
            
        weekDay1 +=1
        
    return data.replace(day=diaVencimento)

##### Pega o dia de negociação anterior
def getDiaDeNegociacaoAnterior(dataReferencia):
  diaAnterior = dataReferencia - datetime.timedelta(days=1)
  feriado = isFeriado(diaAnterior)
  sabado = diaAnterior.weekday() == 5
  domingo = diaAnterior.weekday() == 6

  if(feriado or sabado or domingo):
    return getDiaDeNegociacaoAnterior(diaAnterior)
  else :
    return diaAnterior


##### Monta a URL de CONSULTA de opções
def getUrlConsulta(dataConsulta):
    dataConsulta = dataConsulta.strftime("%Y%m%d")
    
    url = "http://www.b3.com.br/json/"
    url += dataConsulta
    url += "/Posicoes/Empresa/SI_C_OPCPOSABEMP.json" 

    return url

def montaTabela(callsPenultimoDia, callsUltimoDia):
  dados = []
  for opcao in callsPenultimoDia:

    ultimo = callsUltimoDia.get(opcao)
    penultimo = callsPenultimoDia.get(opcao)
    if(not ultimo) or (not penultimo):
      continue

    #[0-codigo, 1-strike, 2-iq, 3-descobertos, 4-total, 5-titular, 6-lancador]
    
    diferencaIQ = ultimo[2] - penultimo[2]
    
    IQ = "%1.2f (%1.2f)" % (ultimo[2], diferencaIQ)


    #DESCOBERTAS
    descobertas = "%1.2f MM " % (ultimo[3]/1000000)
    descobertas = descobertas.rjust(4, " ")

    difDescobertas = (ultimo[3] - penultimo[3])/1000000
    difDescobertas = "%1.2f" % difDescobertas
    difDescobertas = difDescobertas.rjust(5, "+")
    
    descobertas = descobertas + " (" + difDescobertas + ")"


    #TRAVADAS
    travadas = "%1.2f MM " % (ultimo[4]/1000000)
    travadas = travadas.rjust(4, " ")

    difTravadas = (ultimo[4] - penultimo[4])/1000000
    difTravadas = "%1.2f" % difTravadas
    difTravadas = difTravadas.rjust(5, "+")

    travadas = travadas +  " (" + difTravadas + ")"

    total = "%1.2f MM " % (ultimo[5]/1000000)
    total = total.rjust(4, " ")

    difTotal = (ultimo[5] - penultimo[5])/1000000
    difTotal = "%1.2f" % difTotal
    difTotal = difTotal.rjust(5, "+")

    total = total + " (" + difTotal + ")"

    ##print("%8s - R$ %s - IQ:%s - V Descobertas: %s MM - TOTAL %s - Compradores: %s - Lançadores %s" %(ultimo[0], ultimo[1], IQ, descobertas, total, ultimo[5], ultimo[6]))
    ##arquivo.write("%s - R$ %s - IQ:%1.2f (%s) - V Descobertas: %s (%s) MM - Compradores: %d\n" %(ticker, strike, opcao[2], diferenca, descobertas, diferencaDescobertas, opcao[4]))

    dados.append([ultimo[0], ultimo[1], IQ, descobertas, travadas, total , int(ultimo[6]), int(ultimo[7])])

  return dados


### Pega a data do vencimento e os dois últimos dias de negociação

In [5]:
#Data de consulta de hoje e url
hoje =  date.today()
print ("Hoje é", hoje.strftime("%d/%m/%Y"))

#atrasando um dia por causa da B3
#deltaDias = datetime.timedelta(days=1)
#hoje = hoje - deltaDias


dataVencimento = getDataVencimento(hoje)
if (dataVencimento <= hoje):
    diaMesQueVem = hoje + datetime.timedelta(days=30)
    dataVencimento = getDataVencimento(diaMesQueVem)

Hoje é 10/12/2020


In [6]:
vencimento = dataVencimento.strftime("%d/%m/%Y")
print("Vencimento: {}".format(vencimento))

Vencimento: 21/12/2020


In [7]:
ultimoDiaUtil = getDiaDeNegociacaoAnterior(hoje)
dataConsulta = ultimoDiaUtil.strftime("%d/%m/%Y") 
print("Data do último dia de negociação: {}".format(dataConsulta))

urlUltimoDia = getUrlConsulta(ultimoDiaUtil)
print (urlUltimoDia)
print ()

#Data de consulta de ontem url
deltaDias = datetime.timedelta(days=1)
ontem = hoje - deltaDias

penultimoDiaUtil = getDiaDeNegociacaoAnterior(ultimoDiaUtil) 

penultimaDataConsulta = penultimoDiaUtil.strftime("%d/%m/%Y") 
print("Data do peníltimo dia negociação: {}".format(penultimaDataConsulta))

urlPenultimoDiaUtil = getUrlConsulta(penultimoDiaUtil)
print (urlPenultimoDiaUtil)


Data do último dia de negociação: 09/12/2020
http://www.b3.com.br/json/20201209/Posicoes/Empresa/SI_C_OPCPOSABEMP.json

Data do peníltimo dia negociação: 08/12/2020
http://www.b3.com.br/json/20201208/Posicoes/Empresa/SI_C_OPCPOSABEMP.json


### Busca os JSONs dos dois últimos dias e monta as listas

In [8]:
jsonUltimoDia = requests.get(urlUltimoDia)
jsonPenultimoDia = requests.get(urlPenultimoDiaUtil)

In [9]:
def makeFilter(dataVencimento, empresa):
  def filtraOpcoes(item):
    isVencimento = (item['dtVen'] == dataVencimento)
    isEmpresa = (item['mer'] == empresa["ticker"][:4])
    isPN = ( empresa['tipo'] in item['espPap'] )

    if isVencimento and isEmpresa and isPN :
      return True
    return False
  
  return filtraOpcoes

def filtraCalls(item):
  isCall = (item['ser'][4] < 'M')
  if isCall:
    return True
  return False

def filtraPuts(item):
  isPut = (item['ser'][4] >= 'M')
  if isPut:
    return True
  return False

filtroEmpreVenc = makeFilter(dataVencimento.strftime("%Y%m%d"), empresa)

dictJson = json.loads(jsonUltimoDia.text)
lista = dictJson['Empresa'][empresa['letra']]
opcoesUltimoDia = list(filter(filtroEmpreVenc, lista))

dictJson = json.loads(jsonPenultimoDia.text)
lista = dictJson['Empresa'][empresa['letra']]
opcoesPenultimoDia = list(filter(filtroEmpreVenc, lista))

### Separando calls e puts em DataFrame

In [10]:
callsUltimoDia = pd.DataFrame(list(filter(filtraCalls, opcoesUltimoDia)))
putsUltimoDia =  pd.DataFrame(list(filter(filtraPuts, opcoesUltimoDia)))

callsPenultimoDia = pd.DataFrame(list(filter(filtraCalls, opcoesPenultimoDia)))
putsPenultimoDia =  pd.DataFrame(list(filter(filtraPuts, opcoesPenultimoDia)))

### CONVERTENDO O DF com dados da B3 em formato de impressão

In [11]:
def extracaoOpcoes(opts):
  opts.columns = ['serie','exercicio','empresa','coberto','descoberto','titulares','trava','total','lancadores', 'vencimento', 'tipoMercado','mer','espPap']
  opts = opts.sort_values(by=['exercicio'])
  
  hashMap = {}
  for index, row in opts.iterrows():

    titular = row['titulares']
    lancador = row['lancadores']
    
    iq = round(titular/lancador,2) 
    
    descobertos = row['descoberto']

    travadas = row['trava']

    total = row['total']
                
    strike = row['exercicio']
                    
    codigo = row['serie'].strip()
    hashMap[codigo] = [codigo, strike, iq, descobertos, travadas, total, titular, lancador]
  
  return hashMap

### convertendo o DF da B3  
callsUltimoDia = extracaoOpcoes(callsUltimoDia)
callsPenultimoDia = extracaoOpcoes(callsPenultimoDia)

putsUltimoDia =  extracaoOpcoes(putsUltimoDia)
putsPenultimoDia = extracaoOpcoes(putsPenultimoDia)

In [12]:
# escolher a ação desejada
acao = empresa['ticker']+'.SA'
 
# importar dados para o DataFrame
dtInicio = ultimoDiaUtil.strftime("%m/%d/%Y")
#df = dr.DataReader(acao, data_source='yahoo', start=dtInicio)
 
#ultimoPreco = df.iloc[0]
#print(ultimoPreco['Close'])



### ESCOLHER O A QTD DE CALLS DE VENCIMENTO ATUAL QUE VAI APARECER NA TABELA E NOS GRÁFICOS - Não afetas as puts, nem afeta as calls do próximo vencimento - escolher início e fim do intervalo

# CALLS DO VENCIMENTO ATUAL

In [13]:
import openpyxl

inicioCalls = 45
fimCalls = 75

dados = montaTabela(callsPenultimoDia, callsUltimoDia)
dfIQ = pd.DataFrame(dados, columns=['Código', 'Strike', 'IQ', 'Descobertas', 'Travadas', 'Total', 'Titulares', 'Lançadores'])


#display(HTML(dfIQ[1:60].to_html()))
len(dfIQ)

workbook = openpyxl.Workbook()
sheet = workbook.get_sheet_by_name('Sheet')

sheet.title = 'Call'
row = 1
for j in dfIQ['Código']:
    row = row + 1 
    sheet.cell(row = row,column=1).value=j
sheet['A1'] = 'Codigo'  
row = 1
for j in dfIQ['Strike']:
    row = row + 1 
    sheet.cell(row = row,column=2).value=j
sheet['B1'] = 'Strike'
row = 1
for j in dfIQ['Descobertas']:
    row = row + 1 
    sheet.cell(row = row,column=3).value=j
sheet['C1'] = 'Descobertas'
row = 1
for j in dfIQ['Titulares']:
    row = row + 1 
    sheet.cell(row = row,column=4).value=j
sheet['D1'] = 'Titulares'
row = 1
for j in dfIQ['Lançadores']:
    row = row + 1 
    sheet.cell(row = row,column=5).value=j
sheet['E1'] = 'Lançadores'


In [14]:
 workbook.save('BD_opcoes.xlsx')

# PUTS DO VENCIMENTO ATUAL

In [15]:
dados = montaTabela(putsPenultimoDia, putsUltimoDia)
dfIQ = pd.DataFrame(dados, columns=['Código', 'Strike', 'IQ', 'Descobertas', 'Travadas' , 'Total', 'Titulares', 'Lançadores'])
display(HTML(dfIQ[:60].to_html()))
callMes = len(dfIQ)

,Código,Strike,IQ,Descobertas,Travadas,Total,Titulares,Lançadores
0,PETRX656,6.56,8.50 (0.00),1.04 MM (+0.00),0.77 MM (+0.00),1.81 MM (+0.00),119,14
1,PETRX756,7.56,10.38 (0.00),0.15 MM (+0.00),0.67 MM (+0.00),0.82 MM (+0.00),83,8
2,PETRX856,8.56,8.62 (0.00),0.39 MM (+0.00),0.82 MM (+0.00),1.21 MM (+0.00),112,13
3,PETRX956,9.56,12.94 (0.00),1.76 MM (+0.00),1.34 MM (+0.00),3.09 MM (+0.00),233,18
4,PETRX106,10.61,21.13 (0.00),2.83 MM (+0.00),2.34 MM (-0.00),5.17 MM (+0.00),486,23
5,PETRX108,10.86,2.00 (0.00),0.03 MM (+0.00),0.00 MM (+0.00),0.03 MM (+0.00),2,1
6,PETRX116,11.61,14.50 (0.00),9.69 MM (+0.00),0.45 MM (+0.00),10.14 MM (+0.00),435,30
7,PETRX65,11.86,1.50 (0.00),0.33 MM (+0.00),0.00 MM (+0.00),0.33 MM (+0.00),6,4
8,PETRX60,12.11,1.12 (0.00),0.49 MM (+0.00),0.00 MM (+0.00),0.49 MM (+0.00),9,8
9,PETRX88,12.36,0.80 (0.00),0.14 MM (+0.00),0.00 MM (+0.00),0.14 MM (+0.00),4,5


### Buscando o próximo vencimento

In [16]:
proximoVencimento = getDataVencimento(dataVencimento + timedelta(dataVencimento.day+1))
print ("Próximo vencimento é em: ", proximoVencimento.strftime("%d/%m/%Y"))

filtroEmpreVencProximo = makeFilter(proximoVencimento.strftime("%Y%m%d"), empresa)

dictJson = json.loads(jsonUltimoDia.text)
lista = dictJson['Empresa'][empresa['letra']]
opcoesUltimoDia = list(filter(filtroEmpreVencProximo, lista))

dictJson = json.loads(jsonPenultimoDia.text)
lista = dictJson['Empresa'][empresa['letra']]
opcoesPenultimoDia = list(filter(filtroEmpreVencProximo, lista))

Próximo vencimento é em:  18/01/2021


In [17]:
callsPVUltimoDia = pd.DataFrame(list(filter(filtraCalls, opcoesUltimoDia)))
putsPVUltimoDia =  pd.DataFrame(list(filter(filtraPuts, opcoesUltimoDia)))

callsPVPenultimoDia = pd.DataFrame(list(filter(filtraCalls, opcoesPenultimoDia)))
putsPVPenultimoDia =  pd.DataFrame(list(filter(filtraPuts, opcoesPenultimoDia)))

### convertendo o DF da B3 
callsUltimoDiaProximoVencimento = extracaoOpcoes(callsPVUltimoDia)
callsPenultimoDiaProximoVencimento = extracaoOpcoes(callsPVPenultimoDia)

putsUltimoDiaProximoVencimento =  extracaoOpcoes(putsPVUltimoDia)
putsPenultimoDiaProximoVencimento = extracaoOpcoes(putsPVPenultimoDia)

# CALLS DO PRÓXIMO VENCIMENTO

In [23]:
dados = montaTabela(callsPenultimoDiaProximoVencimento, callsUltimoDiaProximoVencimento)
dfIQ = pd.DataFrame(dados, columns=['Código', 'Strike', 'IQ', 'Descobertas', 'Travadas' ,'Total', 'Titulares', 'Lançadores'])
#display(HTML(dfIQ[1:60].to_html()))
len(dfIQ)

row = (callMes + 1)
for j in dfIQ['Código']:
    row = row + 1 
    sheet.cell(row = row,column=1).value=j
sheet['A1'] = 'Codigo'  
row = (callMes + 1)
for j in dfIQ['Strike']:
    row = row + 1 
    sheet.cell(row = row,column=2).value=j
sheet['B1'] = 'Strike'
row = (callMes + 1)
for j in dfIQ['Descobertas']:
    row = row + 1 
    sheet.cell(row = row,column=3).value=j
sheet['C1'] = 'Descobertas'
row = (callMes + 1)
for j in dfIQ['Titulares']:
    row = row + 1 
    sheet.cell(row = row,column=4).value=j
sheet['D1'] = 'Titulares'
row = (callMes + 1)
for j in dfIQ['Lançadores']:
    row = row + 1 
    sheet.cell(row = row,column=5).value=j
sheet['E1'] = 'Lançadores'

In [24]:
workbook.save('BD_opcoes.xlsx')

# PUTS DO PRÓXIMO VENCIMENTO

In [20]:
dados = montaTabela(putsPenultimoDiaProximoVencimento, putsUltimoDiaProximoVencimento)
dfIQ = pd.DataFrame(dados, columns=['Código', 'Strike', 'IQ', 'Descobertas', 'Travadas', 'Total', 'Titulares', 'Lançadores'])
#display(HTML(dfIQ[1:60].to_html()))
len(dfIQ)

81